## Land use and land cover

In [29]:
#   last observed land use type - given secondary ages, get
# if age is 1 in 2020, it was not forest in 2019
# which means I want category in 2019
# if age is 35 in 2020, it was not forest in 1985
# which means I want category in 1985
lulc_masked = lulc.updateMask(age)

last_LU = ee.Image().constant(0).byte().rename('last_LU')

for i in range(first_year + 1, last_year):  # 1986-2020
  year = f'classification_{i}'
  age_mask = age.eq(last_year-i); #keep only the pixels with age equivalent to the correspondent year
  last_LU_observation = lulc.select(year).updateMask(age_mask) #keeps only land use classes of the year before abandonment
  last_LU = last_LU.add(last_LU_observation)

In [30]:
LU_sum = ee.Image().byte()

for val in LU_index:
  lulc_val = lulc_masked.eq(val)
  lulc_val_mask = lulc_val.mask()
  num_cells = lulc_val.reduce(ee.Reducer.sum()).rename(f'lulc_sum_{val}')
  LU_sum = LU_sum.addBands(num_cells)

LU_sum = LU_sum.slice(1).rename('LU_sum')

# Map = geemap.Map(center=[-10, -40], zoom=4)
# vizpar_age = {'min': 1, 'max': last_year - first_year, 'palette': ['blue', 'red']}  # Blue for 0, Red for 1
#vizpar_lulc = {'min': 1, 'max': max(LU_index), 'palette': lulc_palette}
# Map.addLayer(age, vizpar_age, "ages")
# Map.addLayer(num_cells, vizpar_age, "lulc_masked")
# Map
#img_export = img_export + [last_LU, LU_sum]

### Fire
Note that fire has different transform than lulc, so the projection will need to be adjusted when exporting.

In [31]:
# fire has the value of the land use type that burned.
# Transforming into a fire mask:
fire = fire.gt(0)
num_fires = fire.reduce(ee.Reducer.sum()).rename('num_fires')

# get fire frequency data from Mapbiomas - double check it.
# fire_freq = ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-fire-frequency-1").clip(roi)
# fire_freq = fire_freq.select('fire_frequency_1985_2020')

# how many years ago was each fire? #############################
# Get the number of bands
num_bands = fire.bandNames().size()
# Create a sequence of numbers from 1 to num_bands
years_ago = ee.List.sequence(1, num_bands)
years_ago = years_ago.reverse()

# # # Map over the image and set values based on the band index
constant_images = ee.ImageCollection.fromImages(
    years_ago.map(lambda year: ee.Image.constant(year))).toBands()

time_since_all_fires = fire.multiply(constant_images)

old_names = time_since_all_fires.bandNames().getInfo()
new_names = [name.replace('burned_coverage', 'time_since_fire') for name in old_names]
time_since_all_fires = time_since_all_fires.select(old_names).rename(new_names)

# how many years ago was the LAST fire? #############################
last_fire = time_since_all_fires.reduce(ee.Reducer.lastNonNull()).rename('last_fire')

# Map = geemap.Map(center=[-10, -40], zoom=4)
# Map.addLayer(num_fires, {'min':0, 'max':35}, 'num_fires')
# Map

# img_export = img_export + [num_fires, time_since_all_fires, last_fire]